In [59]:
def Lperm_solve_CSR(L_val, L_colval, L_rowptr, b):
    n=len(L_rowptr)-1
    row_count=[]
    for i in range(n): #O(n) subtractions
        row_count.append(L_colval[L_rowptr[i+1]-1])
    #row_count contains indices of last non-zero entries in each row
    #note row_count also reveals the permutation needed to make L lower triangular
    #recover the correct ordering of rows, L_rowptr[perm[i]] is the starting point of row of length i
    perm=[0 for i in range(n)]
    for i in range(n): 
        perm[row_count[i]]=i
    #permute vector b as well to recover the right b to use
    permed_b=[]
    for i in range(n):
        permed_b.append(b[perm[i]])
    #Until this point no flops needed
    #What is left is simply sparse forward substitution
    for i in range(n):
        row_ind=perm[i]
        start=L_rowptr[row_ind]
        end=L_rowptr[row_ind+1]-1 #note we skip diagonals
        for j in range(start,end): 
            #This loop only goes through non-zero entries in the strictly lower triangular part of L
            #Thus it has N-n iteration
            permed_b[i]-=L_val[j]*permed_b[L_colval[j]] #1 multiplicatioon and 1 substractioon
    return permed_b
    

In [62]:
def run_Lperm_test():
    import numpy as np
    import random 
    import scipy.sparse
    n=64
    for k in range(128):
        perm=[i+1 for i in range(n)]
        for i in range(1,n):
            j=i+1+int(random.random()*(n-i))
            assert((i<j) and (j<=n))
            p_tmp=perm[i-1]
            perm[i-1]=perm[j-1]
            perm[j-1]=p_tmp
        L=np.zeros([n,n])
        p=random.random()
        for i in range(1,n+1):
            for j in range(1,i):
                if random.random()>p:
                    L[perm[i-1]-1,j-1]=1.0+(7*i*j)%9
            L[perm[i-1]-1,i-1]=1.0
        x=[i+1 for i in range(n)]
        x_vec=np.reshape(x,(n,1))
        b=np.matmul(L,x_vec)[:,0]
        L_sparse=scipy.sparse.csr_matrix(L)
        Lp=L_sparse.data.tolist()
        Lj=L_sparse.indices.tolist()
        Lx=L_sparse.indptr.tolist()
        assert(x==Lperm_solve_CSR(Lp,Lj,Lx,b))
    print("test successful!")

In [63]:
run_Lperm_test()

test successful!
